# Advanced Trader Behavior vs Market Sentiment Analysis

## Load Libraries

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


## Load Data

In [ ]:

fear = pd.read_csv('data/fear_greed_index.csv')
trades = pd.read_csv('data/historical_data.csv')
fear['Date'] = pd.to_datetime(fear['Date'])
trades['time'] = pd.to_datetime(trades['time'])


## Feature Engineering

In [ ]:

trades['date'] = trades['time'].dt.date
daily = trades.groupby(['account','date']).agg({
'closedPnL':'sum',
'size':'mean',
'leverage':'mean'
}).reset_index()

fear['date'] = fear['Date'].dt.date
df = daily.merge(fear[['date','Classification']], on='date', how='left')


## Sentiment Impact

In [ ]:

df.groupby('Classification')['closedPnL'].mean().plot(kind='bar')
plt.title('Average PnL by Sentiment')
plt.show()


## Trader Segmentation

In [ ]:

df['segment'] = pd.qcut(df['leverage'],3,labels=['Low','Medium','High'])
df.groupby('segment')['closedPnL'].mean()


## Simple Predictive Model

In [ ]:

df['profit_flag']=(df['closedPnL']>0).astype(int)
X=df[['size','leverage']].fillna(0)
y=df['profit_flag']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
model=RandomForestClassifier()
model.fit(X_train,y_train)
pred=model.predict(X_test)
print(classification_report(y_test,pred))
